<a href="https://colab.research.google.com/github/AnIsAsPe/ElementosEstadisticaMachineLearningPython/blob/master/Semana%204/4_2%20Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<html>
<header><title>Curso Fundamentos Técnicos y Aplicaciones de ML- Lección 16</title></header>
<body>
<h1>Principios de Web Scrapping</h1>
<p>Notebook con dos ejemplos de web scraping usando request y BeautifulSoup</p>
</body>
</html>

# Cargar Bibliotecas

In [19]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup  
import requests


## Ejemplo 1 Articulos Toward Data Science en un día

 Articulos de un solo día

In [3]:
# url del que bajaremos la información
url = 'https://towardsdatascience.com/archive/2020/01/10'

#usamos requests para guardar la información de la url
respuesta = requests.get(url)

existen páginas que solicitan ```headers```, que nos identifican (sistema operativo, explorador web, etc), aunque desde Python normalmente se omiten.



con el atributo ```.status_code``` podemos revisar si todo está bien.
Si obtenemos 200, todo está bien, si en cambio obtenemos 404 (el famoso 404) u otro número en el rango de los cuatrocientos o quinientos, todo está mal.

In [4]:
respuesta.status_code 

200

el atributo  ```.text```  guarda la información de la página

In [5]:
respuesta.text

'<!DOCTYPE html><html xmlns:cc="http://creativecommons.org/ns#"><head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# medium-com: http://ogp.me/ns/fb/medium-com#"><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0, viewport-fit=contain"><title>All stories published by Towards Data Science on January 10, 2020</title><link rel="canonical" href="https://towardsdatascience.com/archive/2020/01/10"><meta name="robots" content="index,follow"><meta name="title" content="All stories published by Towards Data Science on January 10, 2020"><meta name="referrer" content="unsafe-url"><meta name="description" content="Read all stories published by Towards Data Science on January 10, 2020. A Medium publication sharing concepts, ideas, and codes."><meta name="theme-color" content="#000000"><meta property="og:title" content="All stories published by Towards Data Science on January 10, 2020"><meta property="twitte

 la libría BeautifulSoup nos ayuda a entender el código HTLM y nos permite haer consultas de manera sencilla

In [8]:
# Analizamos (parse) la información del HTLM de la página utilizando BeautifulSoup
soup = BeautifulSoup(respuesta.text, 'html.parser')


El metodo ``` .find_all()```  permite buscar en el árbol HTML elementos especificos y los devuelve en una lista



In [20]:
for art in soup.find_all('div', class_='streamItem streamItem--postPreview js-streamItem'):
      print(art.find('h3').text)

PySpark and SparkSQL Basics
Predicting Used Car Prices with Machine Learning Techniques
How to pass the Facebook Data Science interview
Most Effective Way To Implement Radial Basis Function Neural Network for Classification Problem
Top features of Pandas 1.0
Training, validation, and test phases in AI — explained in a way you’ll never forget
Deploy MLflow with docker compose
Customer Churn Analysis
Predictive Modelers’ Guide To Choosing The Best Fit Regression Model for Beginners
Translate a Pandas data frame using googletrans library
How I Made A ChromeBook My Data Science Daily Driver
Best Spyder Alternative — VS Code
Getting Started with Data Analysis on AWS
Edge AI
Detecting real-time and unsupervised anomalies in streaming data: a starting point
Achieving Stationarity With Time Series Data
Beautiful Code
Reinforcement Learning Policy for Developers
Linear Regression Explained in 5 Minutes
Breaking Down Goodreads Dataset using Python
How to Code Inheritance in Java — Beginner’s Tut

extraer:
 1. la URL del autor, desde donde podemos extrer luego el nombre de usuario del autor
 2. El tiempo de lectura
 3. El titulo y subtitulo del articulo
 4. El n´mero de aplausos y respuestas
 5. La URL del botón "Read more"


In [21]:
articulos = soup.find_all('div', class_='streamItem streamItem--postPreview js-streamItem')

articulos_dic = {}
for i, art in zip(range(len(articulos)), articulos):
    cada_articulo = {}

    # 1. URL del autor
    apartado_autor = art.find('div', class_='postMetaInline u-floatLeft u-sm-maxWidthFullWidth')
    cada_articulo['url_autor'] = apartado_autor.find('a')['href']

    # 2. El tiempo de lectura
    try:
        cada_articulo['reading_time'] = apartado_autor.find('span', class_='readingTime')['title']
    except:
        continue

    # 3. El titulo y el subtitulo

    cada_articulo['titulo'] = art.find('h3').text if art.find('h3') else '-'
    cada_articulo['subtitulo']  = art.find('h4').text if art.find('h4') else '-'

    # 4. aplausos
    if art.find('button', class_='button button--chromeless u-baseColor--buttonNormal js-multirecommendCountButton u-disablePointerEvents'):
        cada_articulo['aplausos']  = art.find('button', class_='button button--chromeless u-baseColor--buttonNormal js-multirecommendCountButton u-disablePointerEvents').text    
    else:
        cada_articulo['aplausos'] = 0
    
    # 4.respuestas

    if art.find('a',  class_='button button--chromeless u-baseColor--buttonNormal'):
        cada_articulo['respuestas'] = art.find('a',  class_='button button--chromeless u-baseColor--buttonNormal').text
    else:
        cada_articulo['respuestas']= '0 responses'
    
    # url leer más
    cada_articulo['url leer mas'] = art.find('a', class_='button button--smaller button--chromeless u-baseColor--buttonNormal')['href']
    articulos_dic[i] =cada_articulo

In [22]:
articulos_dic

{0: {'aplausos': '697',
  'reading_time': '8 min read',
  'respuestas': '9 responses',
  'subtitulo': 'Big Data Implementation with\xa0PySpark',
  'titulo': 'PySpark and SparkSQL\xa0Basics',
  'url leer mas': 'https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53?source=collection_archive---------0-----------------------',
  'url_autor': 'https://towardsdatascience.com/@pinarersoy'},
 1: {'aplausos': '302',
  'reading_time': '20 min read',
  'respuestas': '1 response',
  'subtitulo': 'Comparing Performance of Five Different ML…',
  'titulo': 'Predicting Used Car Prices with Machine Learning Techniques',
  'url leer mas': 'https://towardsdatascience.com/predicting-used-car-prices-with-machine-learning-techniques-8a9d8313952?source=collection_archive---------1-----------------------',
  'url_autor': 'https://towardsdatascience.com/@enesgokce'},
 2: {'aplausos': '976',
  'reading_time': '5 min read',
  'respuestas': '1 response',
  'subtitulo': '4 skills you need to ace 

In [23]:
articulos = pd.DataFrame(articulos_dic).transpose()
print(articulos.shape)
articulos

(35, 7)


,url_autor,reading_time,titulo,subtitulo,aplausos,respuestas,url leer mas
0,https://towardsdatascience.com/@pinarersoy,8 min read,PySpark and SparkSQL Basics,Big Data Implementation with PySpark,697,9 responses,https://towardsdatascience.com/pyspark-and-spa...
1,https://towardsdatascience.com/@enesgokce,20 min read,Predicting Used Car Prices with Machine Learni...,Comparing Performance of Five Different ML…,302,1 response,https://towardsdatascience.com/predicting-used...
2,https://towardsdatascience.com/@tjwaterman99,5 min read,How to pass the Facebook Data Science interview,4 skills you need to ace the interview loop,976,1 response,https://towardsdatascience.com/how-to-pass-the...
3,https://towardsdatascience.com/@tarlanahad,6 min read,Most Effective Way To Implement Radial Basis F...,-,185,1 response,https://towardsdatascience.com/most-effective-...
4,https://towardsdatascience.com/@tjwaterman99,3 min read,Top features of Pandas 1.0,New improvements you can start using today,1.7K,5 responses,https://towardsdatascience.com/top-features-of...
5,https://towardsdatascience.com/@kozyrkov,6 min read,"Training, validation, and test phases in AI — ...",-,943,3 responses,https://towardsdatascience.com/training-valida...
6,https://towardsdatascience.com/@GuAndroz,6 min read,Deploy MLflow with docker compose,When in the process of building and training m...,70,3 responses,https://towardsdatascience.com/deploy-mlflow-w...
7,https://towardsdatascience.com/@anilak1978_94291,9 min read,Customer Churn Analysis,Brief Overview of Customer Churn Analysis and ...,39,1 response,https://towardsdatascience.com/customer-churn-...
8,https://towardsdatascience.com/@freemangoja,9 min read,Predictive Modelers’ Guide To Choosing The Bes...,-,114,0 responses,https://towardsdatascience.com/predictive-mode...
9,https://towardsdatascience.com/@amandaiglesias...,4 min read,Translate a Pandas data frame using googletran...,-,90,6 responses,https://towardsdatascience.com/translate-a-pan...


In [24]:
articulos.to_csv('articulos.csv', index=False)

## Ejemplo 2. Mercado Libre, precios de guitarras eléctricas

In [114]:


def precios_guitarras():
    url = "https://listado.mercadolibre.com.mx/guitarra-electrica#D[A:guitarra%20electrica]"
    res_ml = requests.get(url)
    soup = BeautifulSoup(res_ml.text, 'html.parser')
    precios = []
    for item in soup.find_all('div', class_='ui-search-price ui-search-price--size-medium ui-search-item__group__element'):
      precio = item.find('div', class_='ui-search-price__second-line').text
      precio = precio.replace('$','')
      precio = precio.replace(',','')
      precios.append(int(precio)) 

    return precios

precios = precios_guitarras()
np.mean(precios)

7795.0